<a href="https://colab.research.google.com/github/mariasald/avd-2024-01/blob/main/Atividade_Pr%C3%A1tica_4_O_Problema_do_Transiente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Alunos**

* Gabriel César Tavares Ferreira - 21854868
* Maria Luiza Saldanha - 22153140

#**Bibliotecas Utilizadas**

#**Parte 1**

* Estimar $\bar{X}(n)$, o tempo médio de espera em uma fila M/M/1 (n=103,λ=9,μ=10).
* Calcular o valor teórico.
* Computar o viés (bias): B=X¯(n)−E[X].
* Repetir a simulação r vezes (r=30).
* Anotar os resultados em uma tabela.
* Analisar os resultados.

#**Parte 2**

* Simular o tempo médio de espera em uma fila M/M/1 (λ=9,μ=10).
* Eliminar o transiente usando as heurísticas de (a) Conway e (b) Fishman.
* Após eliminar o transiente, coletar n valores no estado estacionário ( n=103).
* Calcular X¯(n), o tempo médio de espera na fila e o viés (B).
* Repetir a simulação r vezes (r=30).
* Mostrar os resultados em uma tabela.
* Analisar os resultados.

#**Parte 3**

* Executar uma simulação de horizonte infinito da fila M/M/1.
* Eliminar o transiente usando a heurística MSER-5Y.
* Estimar o tempo médio de espera na fila e o respectivo intervalo de confiança de 95%.
* A regra de parada é a precisão relativa de 5%.
* Cenário    I:  λ=7 clientes por segundo; μ=10 clientes por segundo.
* Cenário   II:  λ=8 clientes por segundo; μ=10 clientes por segundo.
* Cenário  III:  λ=9 clientes por segundo; μ=10 clientes por segundo.
* Cenário IV:  λ=9,5 clientes por segundo; μ=10 clientes por segundo.
* Plotar um gráfico comparativo dos quatro cenários.
* Plotar uma linha horizontal para o valor teórico esperado em cada cenário.
* Analisar os resultados.